### Data Description 
The data containss metadata for all 45000 movie listed in FullMovieLens Dataset. The movies_metadata file contains information such as revenue, budget, release date, average vote,... for each movie. The credit data contains information like cast, crew,... 

###  Goal
Creating a regression model to predict the avergae vote on each movie.

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from collections import defaultdict

import pandasql 
from pandasql import sqldf
from six.moves import cPickle as pickle
from ast import literal_eval



from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import BaseEstimator, TransformerMixin


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import StackingRegressor



from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR


from sklearn.metrics import mean_squared_error



pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-65114ea98385>:50: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
pd.options.display.float_format = "{:,.2f}".format

In [3]:
pysqldf = lambda q: sqldf(q,globals())

### 1 Importing  and cleaning data


In [4]:
metadata=pd.read_csv('input/movies_metadata.csv')

/home/zahra/myenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
credits=pd.read_csv('input/credits.csv')

In [6]:
df_m=metadata.copy()

df_c=credits.copy()

In [7]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

#### 1.1 reducing metadata data to movies  with rating in top 5%

In [8]:
#we define the following two functions for the purpose of data-cleaning.
def isfloat(x):
    try:
        a = float(x)
    except (TypeError, ValueError):
        return False
    else:
        return True
    
def isint(x):
    try:
        a = int(x)
    except (TypeError, ValueError):
        return False
    else:
        return True    

In [9]:
#we restrict ourseleves moview with non-null id 
df_m=df_m[df_m['id'].notnull()]

In [10]:
df_m['id']=df_m[df_m['id'].apply(lambda x:isint(x))==True]['id'].astype(int)

In [11]:
df_m['vote_count'] = df_m[df_m['vote_count'].notnull()]['vote_count'].astype('int')

In [12]:
m = df_m['vote_count'].quantile(0.95)
print(m)

434.0


In [13]:
df_m=df_m[(df_m['vote_count'] >= m)]

In [14]:
df_m.shape

(2274, 24)

#### 1.2 Saving dataframe for creating recommender model

In [15]:
df_m.shape

(2274, 24)

In [16]:
#saving the dataframe for recommender system
df_m.to_csv('movies_metadata_updated.csv')

#### 1.3 Tranforming movie genre to a number

We only consider the genres which appered more than 10 times. We then consider a hashing map for the genres column which send the list of genres for each movie to a vector of length of the total number of genes. Then we use pca to reduce the dimension of these vectors to 1 and put this value in a new column called genre_final.

In [17]:
df_m['genres'] = df_m['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [18]:

s = df_m.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

In [19]:
s.value_counts()

Drama              900
Comedy             743
Action             723
Thriller           709
Adventure          565
Crime              375
Science Fiction    369
Romance            334
Fantasy            329
Family             304
Horror             249
Mystery            207
Animation          200
War                79 
History            77 
Music              61 
Western            29 
Documentary        6  
TV Movie           2  
dtype: int64

In [20]:
genres=list(s.value_counts()[s.value_counts()>10].index)
len_gen=len(genres)
Dict_genre=dict(zip(genres,range(len_gen)))

In [21]:
Dict_genre


{'Drama': 0,
 'Comedy': 1,
 'Action': 2,
 'Thriller': 3,
 'Adventure': 4,
 'Crime': 5,
 'Science Fiction': 6,
 'Romance': 7,
 'Fantasy': 8,
 'Family': 9,
 'Horror': 10,
 'Mystery': 11,
 'Animation': 12,
 'War': 13,
 'History': 14,
 'Music': 15,
 'Western': 16}

In [22]:
def hashing_genres(List):

    vector=np.zeros(len_gen)
    for a in List:
        if a in Dict_genre.keys():
            vector[Dict_genre[a]]=1
    return vector        

In [23]:
for i in range(len_gen):
    df_m["genre_{}".format(i)]=(df_m['genres'].apply(hashing_genres)).apply(lambda x : x[i])
    List_g=[]
for i in range(len_gen):
     List_g.append("genre_{}".format(i))

In [24]:
pca = PCA(n_components=1)
pca.fit(df_m[List_g])
X=pca.transform(df_m[List_g])

In [25]:
df_m['genre_final']=X.reshape((X.shape[0],)).astype(float)

In [26]:
df_m=df_m.drop(List_g,axis=1)

#### 1.4 Deforming release date to year

In [27]:
#we change the values of release date column to the year of release of the movie

df_m.release_date=df_m.release_date[df_m.release_date.notnull()].apply(lambda x: x.split("-")[0]).astype(int)

#### 1.5 Managing null data

In [28]:
# we consider only scalar values for popularity column
df_m['popularity']=df_m.popularity[df_m.popularity.apply(lambda x: isfloat(x))==True].astype('float')

In [29]:
## we consider only scalar values for budget columns
df_m['budget']=df_m.budget[df_m.budget.apply(lambda x: isfloat(x))==True].astype('float')

In [30]:
#we put xx for any movie with no original language specified 
df_m["original_language"]=df_m["original_language"].fillna('xx')

In [31]:
#list(df_m["original_language"].value_counts()[df_m["original_language"].value_counts()>10].index)

In [32]:
# We fill the null vallues for runtime with the median of the runtime for movies with the same original laanguage and revenue
grouped = df_m.groupby(['original_language','revenue'])

In [33]:
df_m.runtime = grouped.runtime.apply(lambda x: x.fillna(x.median()))


#### 1.6 Checking the corrolation matrix

In [34]:
List=['id','title','adult','vote_count', 'runtime','revenue','vote_average','genre_final', 'release_date','popularity','budget','original_language']

In [35]:
df_m_reduced = df_m[(df_m['vote_average'].notnull())][List]


In [36]:
df_m_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2274 entries, 0 to 45014
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2274 non-null   float64
 1   title              2274 non-null   object 
 2   adult              2274 non-null   object 
 3   vote_count         2274 non-null   float64
 4   runtime            2274 non-null   float64
 5   revenue            2274 non-null   float64
 6   vote_average       2274 non-null   float64
 7   genre_final        2274 non-null   float64
 8   release_date       2274 non-null   int64  
 9   popularity         2274 non-null   float64
 10  budget             2274 non-null   float64
 11  original_language  2274 non-null   object 
dtypes: float64(8), int64(1), object(3)
memory usage: 231.0+ KB


In [37]:
df_m_num_attribs=list(df_m_reduced.select_dtypes('float64').columns)

In [38]:
df_m_reduced[df_m_num_attribs].corr()

,id,vote_count,runtime,revenue,vote_average,genre_final,popularity,budget
id,1.00,0.03,-0.03,-0.04,-0.06,-0.01,0.16,-0.04
vote_count,0.03,1.00,0.25,0.70,0.32,0.14,0.40,0.49
runtime,-0.03,0.25,1.00,0.21,0.30,0.18,0.10,0.24
revenue,-0.04,0.70,0.21,1.00,0.05,0.04,0.37,0.70
vote_average,-0.06,0.32,0.30,0.05,1.00,-0.09,0.12,-0.18
genre_final,-0.01,0.14,0.18,0.04,-0.09,1.00,0.05,0.16
popularity,0.16,0.40,0.10,0.37,0.12,0.05,1.00,0.21
budget,-0.04,0.49,0.24,0.70,-0.18,0.16,0.21,1.00


#### 1.7  extracting the driector from the crew column in credits data

In [39]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [40]:
df_c=df_c[df_c['id'].notnull()]

We define the director function to get the director name out of the crew column.

In [41]:
def director(crew):
    for x in crew:
        if x["job"]=='Director':
            return x['name']
        else: return np.nan

In [42]:
df_c['director']=df_c['crew'].apply(literal_eval).apply(director)

In [43]:
#df_c['director'].value_counts()

#### 1.8 extracting the first 3 casts from the cast column

From the task column we only retrieve the name of the first 3 casts. Then we create three new columns for these names.

In [44]:
df_c['cast']=df_c['cast'].apply(literal_eval)

In [45]:
df_c['cast'] = df_c['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df_c['cast'] = df_c['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [46]:
for i in range(3):
    df_c['cast_{}'.format(i)]=df_c['cast'].apply(lambda x: x[i] if len(x)>=i+1 else np.nan)

In [47]:
df_c.columns

Index(['cast', 'crew', 'id', 'director', 'cast_0', 'cast_1', 'cast_2'], dtype='object')

In [48]:
df_c_reduced=df_c[['id','director','cast_0','cast_1','cast_2']]

In [49]:
df_c_reduced=df_c_reduced[(df_c_reduced['director'].notnull())&(df_c_reduced['cast_0'].notnull())&(df_c_reduced['cast_1'].notnull())&(df_c_reduced['cast_2'].notnull())]

#### 1.9 merging reduced metadata and reduced credits data

In [50]:
table=df_m_reduced.merge(df_c_reduced, on='id')

In [51]:
table.id=table.id.astype(object)

In [52]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703 entries, 0 to 702
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 703 non-null    object 
 1   title              703 non-null    object 
 2   adult              703 non-null    object 
 3   vote_count         703 non-null    float64
 4   runtime            703 non-null    float64
 5   revenue            703 non-null    float64
 6   vote_average       703 non-null    float64
 7   genre_final        703 non-null    float64
 8   release_date       703 non-null    int64  
 9   popularity         703 non-null    float64
 10  budget             703 non-null    float64
 11  original_language  703 non-null    object 
 12  director           703 non-null    object 
 13  cast_0             703 non-null    object 
 14  cast_1             703 non-null    object 
 15  cast_2             703 non-null    object 
dtypes: float64(7), int64(1), o

####  1.10 seperating train and test data and use label encoder for columns with string datatype

Seperating label and features.

In [53]:
y=table['vote_average']

In [54]:
X=table.drop(['vote_average'],axis=1)

In [55]:
num_attribs=(list(X.select_dtypes('float64').columns))
cat_attribs=list(X.select_dtypes('object').columns)

In [56]:
X=X[num_attribs+cat_attribs]

In [57]:
print(num_attribs)

['vote_count', 'runtime', 'revenue', 'genre_final', 'popularity', 'budget']


In [58]:
print(cat_attribs)

['id', 'title', 'adult', 'original_language', 'director', 'cast_0', 'cast_1', 'cast_2']


We use lable encoder transformer for categorical data. 

In [59]:
cat_attribs1=['adult', 'original_language', 'director', 'cast_0', 'cast_1', 'cast_2']

In [60]:
le=LabelEncoder()
for x in cat_attribs1:
    X[x]=le.fit_transform(X[x])

In [61]:
X_train_n, X_test_n, y_train, y_test = train_test_split(
   X, y, test_size=0.33, shuffle=True)


In [62]:
X_train_n.shape

(471, 14)

In [63]:
class DataFrameSelector(BaseEstimator, TransformerMixin): 
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names 
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [64]:

num_pipeline = Pipeline([
     ('selector', DataFrameSelector(num_attribs)),
     ('imputer', SimpleImputer(strategy="median")),
     ('std_scaler', StandardScaler()),])

cat_pipeline = Pipeline([
     ('selector', DataFrameSelector(cat_attribs1)),
     ])


full_pipeline = FeatureUnion(transformer_list=[
     ("num_pipeline", num_pipeline),
     ("cat_pipeline", cat_pipeline),])



In [65]:

X_train = full_pipeline.fit_transform(X_train_n)


In [66]:
X_train.shape

(471, 12)

In [67]:
"""X_1=pd.DataFrame(X_train)
result = pd.concat([X_1,y_train], axis=1, join="inner")
result.corr()"""

'X_1=pd.DataFrame(X_train)\nresult = pd.concat([X_1,y_train], axis=1, join="inner")\nresult.corr()'

### 2 Applying Learning algorithm 

#### 2.1 SVR and Tunning parameters

In [68]:
param_grid=[{
            'kernel':['rbf','linear'],
            'C': [ 1,10, 100,1000],
            'epsilon': [0.01,0.1, 1],
            'gamma': [ 0.0001,0.001,  0.1, 1],       
            },
            {
            'kernel':['poly'],
            'C': [ 1,10, 100,1000],
            'epsilon': [0.01,0.1, 1],
            'gamma': [ 0.0001,0.001,  0.1, 1],
            'degree':[3,4,5,7]          
            }]

In [69]:
grid_search_svr= GridSearchCV(
        SVR(max_iter=5000),param_grid=param_grid,
        cv=5,scoring='neg_mean_squared_error', n_jobs=-1)

In [70]:
grid_search_svr.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SVR(max_iter=5000), n_jobs=-1,
             param_grid=[{'C': [1, 10, 100, 1000], 'epsilon': [0.01, 0.1, 1],
                          'gamma': [0.0001, 0.001, 0.1, 1],
                          'kernel': ['rbf', 'linear']},
                         {'C': [1, 10, 100, 1000], 'degree': [3, 4, 5, 7],
                          'epsilon': [0.01, 0.1, 1],
                          'gamma': [0.0001, 0.001, 0.1, 1],
                          'kernel': ['poly']}],
             scoring='neg_mean_squared_error')

In [71]:
grid_search_svr.best_params_

{'C': 1, 'epsilon': 1, 'gamma': 0.1, 'kernel': 'rbf'}

In [72]:
model_svr=grid_search_svr.best_estimator_

In [73]:
"""gsc_svr.best_score_
y_hat=model_svr.predict(X_test)
err_svr=mean_squared_error(y_hat,y_test)"""

'gsc_svr.best_score_\ny_hat=model_svr.predict(X_test)\nerr_svr=mean_squared_error(y_hat,y_test)'

In [74]:
pickle.dump(model_svr, open('svr', 'wb'))

####  2.2 Random Forest Regressor and tunnig parametes

In [75]:
param_grid = {'bootstrap': [True,False],'n_estimators': [50,100,500], 'max_features': [5,9,14]}

forest_reg = RandomForestRegressor()
grid_search_rf = GridSearchCV(forest_reg, param_grid, cv=5,
scoring='neg_mean_squared_error')
grid_search_rf.fit(X_train, y_train)

/home/zahra/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(b

/home/zahra/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(b

/home/zahra/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(b

/home/zahra/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(b

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [True, False], 'max_features': [5, 9, 14],
                         'n_estimators': [50, 100, 500]},
             scoring='neg_mean_squared_error')

In [76]:
grid_search_rf.best_params_

{'bootstrap': True, 'max_features': 5, 'n_estimators': 50}

In [77]:
model_rf=grid_search_rf.best_estimator_

In [78]:
pickle.dump(model_rf, open('rf', 'wb'))

#### 2.3 KNN and tunning parameters 

In [79]:
pca=PCA()
lda=LinearDiscriminantAnalysis()
knn=KNeighborsRegressor()
pipe=Pipeline(steps=[('pca', pca), ('lda', lda),('knn',knn)])

In [80]:
params_grid =[{
    "knn__n_neighbors": [3,5,7],
    "pca__n_components":[2,3],
    "lda":[None]
},
{
    "knn__n_neighbors": [3,5,7],
    "lda__n_components":[2,3],
    "pca":[None]
}    
]

grid_search_knn = GridSearchCV(pipe, param_grid = params_grid, cv=5,
                               scoring='neg_mean_squared_error')


In [81]:
grid_search_knn.fit(X_train,y_train)

/home/zahra/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/zahra/myenv/lib/python3.8/site-packages/joblib/memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "/home/zahra/myenv/lib/python3.8/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transform(X, y, **f

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pca', PCA()),
                                       ('lda', LinearDiscriminantAnalysis()),
                                       ('knn', KNeighborsRegressor())]),
             param_grid=[{'knn__n_neighbors': [3, 5, 7], 'lda': [None],
                          'pca__n_components': [2, 3]},
                         {'knn__n_neighbors': [3, 5, 7],
                          'lda__n_components': [2, 3], 'pca': [None]}],
             scoring='neg_mean_squared_error')

In [82]:
print(grid_search_knn.best_params_)
model_knn=grid_search_knn.best_estimator_
pickle.dump(model_knn, open('knn', 'wb'))


{'knn__n_neighbors': 7, 'lda': None, 'pca__n_components': 3}


#### 2.4  AdaBoost and tunning

In [83]:
pipe = Pipeline([('clf', AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=5))),])

# Add a dict of estimator and estimator related parameters in this list
params_grid = [{
                'clf':(AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=5)),),
                'clf__n_estimators': [50,100,500],
                'clf__learning_rate':[0.0001, 0.001, 0.01, 0.1],
                },
                {
                'clf':(AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=10)),),
                'clf__n_estimators': [50,100,500],
                'clf__learning_rate':[0.0001, 0.001, 0.01, 0.1],
                },
               # {'estimator':[Any_other_estimator_you_want],
               #  'estimator__valid_param_of_your_estimator':[valid_values]

              ]

In [84]:
grid_search_ab = GridSearchCV(pipe, param_grid=params_grid, n_jobs=-1, cv=5, scoring='neg_mean_squared_error')

In [85]:
grid_search_ab.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('clf',
                                        AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=5)))]),
             n_jobs=-1,
             param_grid=[{'clf': (AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=5)),),
                          'clf__learning_rate': [0.0001, 0.001, 0.01, 0.1],
                          'clf__n_estimators': [50, 100, 500]},
                         {'clf': (AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=10),
                                                    learning_rate=0.1,
                                                    n_estimators=500),),
                          'clf__learning_rate': [0.0001, 0.001, 0.01, 0.1],
                          'clf__n_estimators': [50, 100, 500]}],
             scoring='neg_mean_squared_error')

In [86]:
print(grid_search_ab.best_params_)
model_ab=grid_search_ab.best_estimator_
pickle.dump(model_ab, open('ab', 'wb'))


{'clf': AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=10),
                  learning_rate=0.1, n_estimators=500), 'clf__learning_rate': 0.1, 'clf__n_estimators': 500}


#### 2.5 Checking the errors of the above four models

In [87]:
Dict={"SVR":grid_search_svr,"Random_Forest":grid_search_rf,"KNN":grid_search_knn,"AdaBoost":grid_search_ab}
best_score=[]
for key,value in Dict.items():
    best_score.append((key,np.round(-value.best_score_,2)))
    
print(best_score)    

[('SVR', 0.65), ('Random_Forest', 0.37), ('KNN', 0.75), ('AdaBoost', 0.38)]


#### 2.6 Stacking

In [88]:
def evaluate_model(model, X, y):
    scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, error_score='raise')
    return np.mean(-scores)

In [89]:
estimators=[('knn',model_knn), ('rf',model_rf),('svr',model_svr),('ab',model_ab)]
stacking = StackingRegressor(estimators = estimators,final_estimator=LinearRegression())

In [90]:
stacking_score=evaluate_model(stacking,X_train,y_train)

In [91]:
print(np.round(stacking_score,2))

0.39


In [92]:
pickle.dump(stacking, open('stacking', 'wb'))

#### 2.7 Test error

In [93]:
model_final=stacking.fit(X_train,y_train)

In [94]:
test_error=mean_squared_error(model_final.predict(full_pipeline.transform(X_test_n)),y_test)

In [95]:
test_error

0.342177984733333

In [96]:
X_test_n['title']

430    Cars 2                                     
554    The Best of Me                             
584    Midnight Special                           
261    What Lies Beneath                          
43     True Romance                               
89     A Clockwork Orange                         
195    Rocky V                                    
266    Hellraiser                                 
103    La Femme Nikita                            
75     Dial M for Murder                          
139    Boogie Nights                              
235    Sister Act                                 
388    Letters from Iwo Jima                      
350    Terminator 3: Rise of the Machines         
55     The Cable Guy                              
93     Return of the Jedi                         
667    Bad Moms                                   
284    The Mexican                                
197    National Lampoon's Christmas Vacation      
23     Star Wars               

In [98]:
model_final.predict(full_pipeline.transform(X_test_n[X_test_n['title']=='Star Wars']))

array([7.92177309])

In [99]:
X_test_n[X_test_n['title']=='Star Wars'].index

Int64Index([23], dtype='int64')

In [101]:
y_test[23]

8.1

### 3 Conclusion

Between SVR, Random Forest, Knn, Adaboost and Stacking model, Adaboost model has the minimum error 0.38 and therefore more approriate model for vote average. 